# 2D cross section in a plant tissue

This notebook contains the Finite Element Modeling (FEM) analysis code used to explore stress and strain distribution in 2D mesh of a cross section of a plant tissue: hexagnal cells snapped on concentric circles.

> **NOTE**
> The notebook should be run within a *hypocot_env* environment.

Cell-cell adhesion plays a critical role in tissue integrity and controlled separation during development. Here, we investigate how adhesion mechanisms may take place under tension and turgor pressure within the "adhesive layer model".

### Load libraries and dependencies

All bvpy functions and classes that are related to this notebook will be loaded by sourcing lib_fun.

> **Important**
> The path to the "input", "output" functions (io_function) is relative to where the notebook was open.
> In this case the notebook was open from the notebook folder.

In [ ]:
import sys
import os

# Add the directory where function.py is located to the Python path
sys.path.append(os.path.expanduser("../io_function/"))

from lib_fun import *

### Generating the Mesh

Once the required libraries are loaded, the next step is to generate the mesh.

> **Note**  
> - The `generate_mesh()` function uses **GMSH** to create `.msh` files from `.geo` files, which define the physical surface attributes.  
> - The `CustomDomainGmsh()` reads the `.msh` files and converts the data into a format compatible with **bvpy**.

In [ ]:
mesh_file = '../../data/in/Orga_cross.geo'
initial_scale = 0.1
generate_mesh(mesh_file, initial_scale)
mesh_path = '../../data/in/Orga_cross.msh'
cd = CustomDomainGmsh(fname=mesh_path)

### Visualizing the Mesh

To ensure the mesh has been correctly loaded and interpreted, you can visualize it using the following command:


In [ ]:
plot(cd)

### Locate the mesh nodes related to the boundary conditions

The different boundaries are circular and coorspond to the outer part of the epidermis or the inner side of the center cell  

In [ ]:
out_wall = Boundary(f'near((x)*(x)+(y)*(y), 21.2*21.2, 4.5)')
all_borders = Boundary('all')
center = Boundary(f'near((x)*(x)+(y)*(y), 3,3)')
inner_border = all_borders  & ~ out_wall

### Setting Boundary Conditions and Material Properties

In the following cell, we define the boundary conditions, material properties, and the governing equations for the simulation. These parameters control how the mesh responds to external forces, displacements, and internal material properties.

#### Boundary Conditions
- **Turgor Pressure (`TP`)**: A normal force applied to the inner boundary of the mesh to simulate the internal pressure within cells.
  
The boundary conditions are implemented using:
- `NormalNeumann`: Applies the turgor pressure as a normal force.  
- `dirichlet`: Fixes specific displacement values at defined boundaries.

#### Material Properties
- **Poisson's Ratio (`poiss`)**: Specifies the material's lateral expansion relative to its longitudinal compression.  
- **Young's Modulus (`young_values_by_labels`)**: Defines the stiffness of each subdomain, with values assigned based on the subdomain label.  

#### Governing Equations
- The **StVenantKirchoffPotential** defines the strain energy density function of a neo-hookean material. It is then attached to a hyperelastic variational form **HyperElasticForm**. This is essential for simulating realistic deformations under stress and strain.
- Plane stress conditions are assumed for this 2D analysis.  

#### Nonlinear Boundary Value Problem (BVP)
The `BVP` class is used to set up and solve the system, combining the domain, variational formulation, and boundary conditions into a single framework. The `nl_inflation` object encapsulates this setup for further analysis.


In [ ]:
# Boundary conditions
TP = 0.5
#bc = [dirichlet([0., 0. , 0.], boundary = out_wall)]
bc = [NormalNeumann(val=-TP, boundary=inner_border),
     dirichlet([0., 0. , 0.], boundary = center)] #NormalNeumann(val=-TP,

elastic_potential = StVenantKirchoffPotential(young=50, poisson=0.3)
heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential, source=[0., 0., 0.],
                                                       plane_stress=True)

cross_turgor = BVP(domain=cd, vform=heterogeneous_Hyperelastic_response, bc=bc)

In [ ]:
tolerance = 1e-10  # Initial tolerance
cross_turgor.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance,
          preconditioner = 'none',
          maximum_iterations = 500)

In [ ]:
filename = f"../../data/out/cross/xdmf/cross_turgor_05.xdmf"
xdmf_save(path=filename, solution=cross_turgor.solution, vform=heterogeneous_Hyperelastic_response)